In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import time
import re
import tensorflow as tf
import tensorflow_hub as hub
import pickle
import os
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import string
%matplotlib inline

os.chdir("/Users/alexdessouky/Desktop/MIDS/w266")

#load training data
twitter_train = pd.read_excel('./w266_final_project/StanceDataset/train.xlsx')

#load test data
twitter_test = pd.read_excel('./w266_final_project/StanceDataset/test.xlsx')

In [2]:
twitter_train.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,3. The tweet is not explicitly expressing opi...,pos


In [3]:
twitter_test.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,He who exalts himself shall be humbled; a...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos
1,RT @prayerbullets: I remove Nehushtan -previou...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,other
2,@Brainman365 @heidtjj @BenjaminLives I have so...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos
3,#God is utterly powerless without Human interv...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@David_Cameron Miracles of #Multiculturalism...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,neg


## Code for loading full Elmo outputs from pickle files and prepping labelled data

In [4]:
# ELMO OUTPUTS
with open("./ELMO Outputs v.2/elmo_train.pickle", "rb") as f:
    train_vectors = pickle.load(f)
    
with open("./ELMO Outputs v.2/elmo_test.pickle", "rb") as f:
    test_vectors = pickle.load(f)

In [5]:
#PREP LABELS
stance_labels_train = np.array(twitter_train['Stance'].apply(lambda x: 
                                                                    2 if x == "FAVOR" else 
                                                                    (1 if x == "NONE" else 0)))

stance_labels_test = np.array(twitter_test['Stance'].apply(lambda x: 
                                                                    2 if x == "FAVOR" else 
                                                                    (1 if x == "NONE" else 0)))

In [6]:
#PREP LABELS FOR NN
train_y = np.zeros(shape = (stance_labels_train.shape[0],3))
train_y[stance_labels_train == 0,0] = 1
train_y[stance_labels_train == 1,1] = 1
train_y[stance_labels_train == 2,2] = 1

test_y = np.zeros(shape = (stance_labels_test.shape[0],3))
test_y[stance_labels_test == 0,0] = 1
test_y[stance_labels_test == 1,1] = 1
test_y[stance_labels_test == 2,2] = 1

In [7]:
test = np.pad(test_vectors['lstm_outputs1'], pad_width = ((0,0),(0,2),(0,0)), mode = 'mean')

In [8]:
test.shape

(1956, 32, 1024)

## Helper function for model performances

In [9]:
def confusion_plot(confusion_matrix, target_names):
    # Plot confusion matrix (via imshow)
    plt.imshow(confusion_matrix, interpolation = "nearest", cmap = plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()

    # Loop through each value of the matrix to add data labels
    width, height = confusion_matrix.shape
    for x in range(width):
        for y in range(height):
            plt.annotate(str(confusion_matrix[x][y]), xy = (y, x), 
                        horizontalalignment = "center",
                        verticalalignment = "center")
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

In [10]:
def lstm_model(activation_function_lstm = 'tanh', 
               activation_function_dense = 'relu',
                optimizer='adam', 
                kernel_initializer=tf.keras.initializers.he_normal(), 
                bias_initializer='zeros'):
    
    # Define Input layer(s)
    input_vectors = tf.keras.layers.Input(shape=(32,1024), name="input_numbers")
    
    #define LSTM layter
    lstm1 = tf.keras.layers.LSTM(32, 
                                 input_shape = (32,1024),
                                 activation=activation_function_lstm,
                                 recurrent_activation=activation_function_lstm,
                                 kernel_initializer=kernel_initializer,
                                 recurrent_initializer=kernel_initializer,
                                 bias_initializer=bias_initializer,
                                 name='lstm1')
    #run lstm layer 1
    lstm_output1 = lstm1(input_vectors)
    
    
    #lstm2 = tf.keras.layers.LSTM(32, 
    #                            input_shape = (32,1024),
    #                            activation=activation_function_lstm,
    #                             recurrent_activation=activation_function_lstm,
    #                             kernel_initializer=kernel_initializer,
    #                             recurrent_initializer=kernel_initializer,
    #                             bias_initializer=bias_initializer,
    #                             name='lstm2')
    
    #lstm_output2 = lstm2(lstm_output1)
    
    
    dense1 = tf.keras.layers.Dense(100, 
                                   activation=activation_function_dense, 
                                   kernel_initializer=kernel_initializer, 
                                   bias_initializer=bias_initializer,
                                   name='dense1')
    
    dense1_output1 = dense1(lstm_output1)
    
    
    
    dense2 = tf.keras.layers.Dense(100, 
                                   activation=activation_function_dense, 
                                   kernel_initializer=kernel_initializer, 
                                   bias_initializer=bias_initializer,
                                   name='dense2') 
    
    dense2_output2 = dense2(dense1_output1)

    
    #define classification layer (softmax since we have 3 classes)
    classification_layer = tf.keras.layers.Dense(3, activation='softmax', name='classification') # layer definition
    classification_output = classification_layer(dense2_output2)   # layer acting on previous layer's output

    # Build and compile model
    lstm_mod = tf.keras.models.Model(input_vectors, classification_output)
    
    lstm_mod.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['categorical_accuracy'])
    
    return lstm_mod

lstm_mod = lstm_model()
lstm_mod.summary()
    
    

W1031 21:52:00.502762 140736218923904 deprecation.py:506] From /Users/alexdessouky/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_numbers (InputLayer)   [(None, 32, 1024)]        0         
_________________________________________________________________
lstm1 (LSTM)                 (None, 32)                135296    
_________________________________________________________________
dense1 (Dense)               (None, 100)               3300      
_________________________________________________________________
dense2 (Dense)               (None, 100)               10100     
_________________________________________________________________
classification (Dense)       (None, 3)                 303       
Total params: 148,999
Trainable params: 148,999
Non-trainable params: 0
_________________________________________________________________


In [11]:
#default_train = train_vectors['default'].reshape(2914, 1, 1024)
#default_test = test_vectors['default'].reshape(1956,1,1024)
lstm_mod.fit(
        train_vectors['lstm_outputs1'],
        train_y,
        epochs=20,
        verbose=1)

Epoch 1/20
2914/2914 [==============================] - 4s 1ms/sample - loss: 1.0643 - categorical_accuracy: 0.4777
Epoch 2/20
2914/2914 [==============================] - 3s 1ms/sample - loss: 1.0542 - categorical_accuracy: 0.4787
Epoch 3/20
2914/2914 [==============================] - 3s 1ms/sample - loss: 1.0538 - categorical_accuracy: 0.4791
Epoch 4/20
2914/2914 [==============================] - 3s 965us/sample - loss: 1.0507 - categorical_accuracy: 0.4791
Epoch 5/20
2914/2914 [==============================] - 3s 972us/sample - loss: 1.0416 - categorical_accuracy: 0.4791
Epoch 6/20
2914/2914 [==============================] - 3s 951us/sample - loss: 1.0213 - categorical_accuracy: 0.4811
Epoch 7/20
2914/2914 [==============================] - 3s 955us/sample - loss: 0.9855 - categorical_accuracy: 0.5051
Epoch 8/20
2914/2914 [==============================] - 3s 1ms/sample - loss: 0.9562 - categorical_accuracy: 0.5340
Epoch 9/20
2914/2914 [==============================] - 3s 913us

In [12]:
outs = lstm_mod.predict(test)
outs

array([[1.0365667e-01, 2.9868767e-01, 5.9765565e-01],
       [9.9814320e-01, 1.0549371e-04, 1.7512687e-03],
       [2.5744680e-01, 4.6923023e-01, 2.7332303e-01],
       ...,
       [6.2988502e-01, 9.4532825e-02, 2.7558219e-01],
       [8.4295434e-01, 5.3453945e-02, 1.0359170e-01],
       [9.8923433e-01, 2.1134689e-03, 8.6522372e-03]], dtype=float32)

In [13]:
outs.shape

(1956, 3)

In [14]:
test_predicts = np.argmax(outs, axis = 1)

In [15]:
stance_labels_test

array([0, 0, 0, ..., 2, 0, 1])

In [16]:
f1_score(stance_labels_test, test_predicts, average = 'macro')

0.3300864822238476